In [0]:
# from keras.datasets import imdb

In [0]:
import pandas as pd

In [0]:
df = pd.read_csv('./movie_small.csv.gz', encoding='utf-8')

In [0]:
df_frases = df['review']

In [0]:
df_label = df['sentiment']

In [6]:
df_frases[3]

'hi for all the people who have seen this wonderful movie im sure thet you would have liked it as much as i. i love the songs once you have seen the show you can sing along as though you are part of the show singing and dancing . dancing and singing. the song ONE is an all time fave musical song too and the strutters at the end with the mirror its so oh you have to watch this one'

In [7]:
df_label.head(n=4)

0    1
1    0
2    0
3    1
Name: sentiment, dtype: int64

In [0]:
num_words = 100

In [9]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [0]:
tokenizer = Tokenizer(num_words=num_words)

In [11]:
df['review'].values[0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

In [0]:
tokenizer.fit_on_texts(df['review'].values)

In [0]:
#tokenizer.word_index

In [0]:
sequences = tokenizer.texts_to_sequences(df['review'].values)

In [15]:
len(sequences[0])

116

In [16]:
len(sequences[1])

125

In [0]:
from keras.preprocessing import sequence

In [0]:
seq_pad = sequence.pad_sequences(sequences,maxlen=100)

In [19]:
len(seq_pad[0])

100

In [0]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Embedding, Dense 

In [0]:
len_seq = 60
seq_pad = sequence.pad_sequences(sequences,maxlen=len_seq)

In [22]:
model = Sequential()

In [23]:
new_din = 32
model.add(Embedding(input_dim=num_words, output_dim=new_din, input_length=len_seq))
model.add(SimpleRNN(units=10, input_shape=(new_din,)))
model.add(Dense(1,activation='sigmoid'))

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 60, 32)            3200      
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 10)                430       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 3,641
Trainable params: 3,641
Non-trainable params: 0
_________________________________________________________________


In [25]:
metrics = ['acc']
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=metrics)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
size_tr = int(0.8*len(seq_pad))
x_tr, y_tr = seq_pad[:size_tr], df_label.values[:size_tr]
x_test, y_test = seq_pad[size_tr:], df_label.values[size_tr:]

In [27]:
x_tr.shape

(800, 60)

In [0]:
from keras.callbacks import EarlyStopping

In [0]:
callback_early = EarlyStopping(min_delta=0.01,patience=20)

In [37]:
history = model.fit(x=x_tr,y=y_tr, batch_size=10, epochs=100,validation_split=0.1,callbacks=[callback_early])

Train on 720 samples, validate on 80 samples
Epoch 1/100
720/720 [==============================] - 3s 5ms/step - loss: 0.0794 - acc: 0.9819 - val_loss: 1.6370 - val_acc: 0.4875
Epoch 2/100
720/720 [==============================] - 3s 5ms/step - loss: 0.0327 - acc: 0.9986 - val_loss: 1.5716 - val_acc: 0.5250
Epoch 3/100
720/720 [==============================] - 4s 5ms/step - loss: 0.0240 - acc: 0.9986 - val_loss: 1.5426 - val_acc: 0.5375
Epoch 4/100
720/720 [==============================] - 3s 5ms/step - loss: 0.0194 - acc: 1.0000 - val_loss: 1.6481 - val_acc: 0.5375
Epoch 5/100
720/720 [==============================] - 3s 5ms/step - loss: 0.0179 - acc: 1.0000 - val_loss: 1.6186 - val_acc: 0.5250
Epoch 6/100
720/720 [==============================] - 3s 5ms/step - loss: 0.0164 - acc: 1.0000 - val_loss: 1.6399 - val_acc: 0.5000
Epoch 7/100
720/720 [==============================] - 3s 5ms/step - loss: 0.0153 - acc: 1.0000 - val_loss: 1.6036 - val_acc: 0.5250
Epoch 8/100
720/720 [===

In [0]:
import matplotlib.pyplot as plt


In [38]:
evalu = model.evaluate(x=x_test,y=y_test)

200/200 [==============================] - 0s 1ms/step


In [39]:
evalu

[1.0877380657196045, 0.565]

In [41]:
history.params

{'batch_size': 10,
 'do_validation': True,
 'epochs': 100,
 'metrics': ['loss', 'acc', 'val_loss', 'val_acc'],
 'samples': 720,
 'steps': None,
 'verbose': 1}